In [1]:
import pandas as pd

In [14]:
import pandas as pd

# Read the TSV file
# Using sep='\t' since it's tab-separated
# Comment '#' to ignore lines starting with #
df = pd.read_csv('/Users/saianoopavunuri/Documents/cs_projects/hackathons/Hacklytics2025/HomoSapiens_binary_hq.txt', sep='\t')

In [27]:
import pandas as pd
import numpy as np
from scipy import sparse

# Get unique proteins
proteins = sorted(list(set(df['Uniprot_A'].unique()) | set(df['Uniprot_B'].unique())))

# Create mapping of protein IDs to matrix indices
protein_to_idx = {protein: idx for idx, protein in enumerate(proteins)}

# Create empty sparse matrix in COO format (most efficient for construction)
rows = []
cols = []
data = []

# Fill the sparse matrix
for _, row in df.iterrows():
    i = protein_to_idx[row['Uniprot_A']]
    j = protein_to_idx[row['Uniprot_B']]
    # Add both directions for undirected graph
    rows.extend([i, j])
    cols.extend([j, i])
    data.extend([1, 1])

# Convert to CSR format (most efficient for matrix operations)
adj_matrix = sparse.csr_matrix((data, (rows, cols)), 
                             shape=(len(proteins), len(proteins)))

# Display basic information
print(f"Number of unique proteins: {len(proteins)}")
print(f"Shape of adjacency matrix: {adj_matrix.shape}")
print(f"Number of non-zero elements: {adj_matrix.nnz}")
print(f"Sparsity: {adj_matrix.nnz/(len(proteins)**2)*100:.2f}%")

# Save in compressed sparse format
sparse.save_npz('protein_adjacency_sparse.npz', adj_matrix)

# To save the protein index mapping for later use
import json
with open('protein_index_mapping.json', 'w') as f:
    json.dump(protein_to_idx, f)

Number of unique proteins: 21137
Shape of adjacency matrix: (21137, 21137)
Number of non-zero elements: 320836
Sparsity: 0.07%
